## RAG

In [ ]:
cd /content/drive/MyDrive/rag_ass2

/content/drive/MyDrive/rag_ass2


In [ ]:
from huggingface_hub import login
import os

In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/rag/uet_vnu.csv")
def join_string(item):
    for i in range(len(item)):
        category, content= item

        final_string = ""
        if category:
            final_string += f"{category}."

        if content:
            content = content.replace("<br>", " ").replace("\n", " ")
            final_string += f" {content}"

    return final_string
df['information'] = df[
    [
     'category',
     'content']
    ].astype(str).apply(join_string, axis=1)
df = df[df['information'].notna()]

In [ ]:
docs = df['information'].tolist()

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
#from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
#embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-m3")
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5",
                                       model_kwargs=model_kwargs)
text_splitter = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile",
)
all_chunked_documents = []
for index, row in df.iterrows():
    text_to_split = row['information']

    chunks = text_splitter.create_documents([text_to_split])

    #print(f"Số lượng chunk ngữ nghĩa được tạo: {len(chunks)}")
    for i, chunk_doc in enumerate(chunks):
        # print(f"  Chunk {i+1} (một phần): \"{chunk_doc.page_content[:100]}...\"")
        all_chunked_documents.append(chunk_doc)
    #print("-" * 30)

print(f"\nĐã xử lý xong. Tổng số chunk được tạo: {len(all_chunked_documents)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Đã xử lý xong. Tổng số chunk được tạo: 545


In [ ]:
for doc in all_chunked_documents:
    doc.page_content = "search_document: " + doc.page_content

In [ ]:
all_chunked_documents[1].page_content

'search_document: Mục tiêu chung Mục tiêu của chương trình Công nghệ thông tin (CNTT) tại Trường ĐHCN, ĐHQGHN là đào tạo nguồn nhân lực cử nhân có trình độ chuyên môn cao về CNTT, có năng lực phát triển, triển khai và vận hành các hệ thống phần mềm và các giải pháp CNTT, có khả năng làm việc trong các doanh nghiệp lớn ở Việt Nam cũng như các nước trong khu vực, đáp ứng nhu cầu của xã hội về nhân lực chất lượng cao trong lĩnh vực Công nghệ thông tin. 2.2. Mục tiêu cụ thể Sinh viên thuộc chương trình CNTT sẽ được trang bị các kiến thức và kỹ năng đáp ứng nhu cầu phát triển và ứng dụng CNTT của xã hội trong xu thế toàn cầu hóa; Sinh viên được đào tạo theo một số định hướng chuyên sâu trong lĩnh vực CNTT, bao gồm: (1) định hướng Công nghệ phần mềm tập trung vào các phương pháp, công cụ và quy trình phát triển, quản lý và bảo trì hệ thống phần mềm. (2) định hướng Hệ thống thông tin trang bị kiến thức về phát triển và vận hành các hệ thống thông tin cho việc quản lý kinh tế, hành chính và dị

In [ ]:
import pickle

# Define the path to save the pickle file
save_path = "/content/drive/MyDrive/rag_ass2/chunked_documents.pkl"

# Save the all_chunked_documents list
with open(save_path, 'wb') as f:
    pickle.dump(all_chunked_documents, f)

print(f"Saved chunked documents to {save_path}")

Saved chunked documents to /content/drive/MyDrive/rag_ass2/chunked_documents.pkl


In [ ]:
import pickle
# Define the path to load the pickle file
load_path = "/content/drive/MyDrive/rag_ass2/chunked_documents.pkl"

# Load the all_chunked_documents list
try:
    with open(load_path, 'rb') as f:
        loaded_all_chunked_documents = pickle.load(f)
    print(f"Loaded chunked documents from {load_path}")
    # Now you can use loaded_all_chunked_documents
except FileNotFoundError:
    print(f"Error: File not found at {load_path}")
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

Loaded chunked documents from /content/drive/MyDrive/rag_ass2/chunked_documents.pkl


In [ ]:
loaded_all_chunked_documents[1].page_content
all_chunked_documents = loaded_all_chunked_documents

In [ ]:
all_chunked_documents[1].page_content

'search_document: Mục tiêu chung Mục tiêu của chương trình Công nghệ thông tin (CNTT) tại Trường ĐHCN, ĐHQGHN là đào tạo nguồn nhân lực cử nhân có trình độ chuyên môn cao về CNTT, có năng lực phát triển, triển khai và vận hành các hệ thống phần mềm và các giải pháp CNTT, có khả năng làm việc trong các doanh nghiệp lớn ở Việt Nam cũng như các nước trong khu vực, đáp ứng nhu cầu của xã hội về nhân lực chất lượng cao trong lĩnh vực Công nghệ thông tin. 2.2. Mục tiêu cụ thể Sinh viên thuộc chương trình CNTT sẽ được trang bị các kiến thức và kỹ năng đáp ứng nhu cầu phát triển và ứng dụng CNTT của xã hội trong xu thế toàn cầu hóa; Sinh viên được đào tạo theo một số định hướng chuyên sâu trong lĩnh vực CNTT, bao gồm: (1) định hướng Công nghệ phần mềm tập trung vào các phương pháp, công cụ và quy trình phát triển, quản lý và bảo trì hệ thống phần mềm. (2) định hướng Hệ thống thông tin trang bị kiến thức về phát triển và vận hành các hệ thống thông tin cho việc quản lý kinh tế, hành chính và dị

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
#embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-m3")
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5",
                                       model_kwargs=model_kwargs)
db = Chroma(
    collection_name="uet_vnu",
    persist_directory="/content/drive/MyDrive/rag_ass2/chroma_langchain_db",  # Where to save data locally, remove if not necessary
)
vectorstore = db.from_documents(all_chunked_documents, embeddings)

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 10})
keyword_retriever = BM25Retriever.from_documents(all_chunked_documents)
keyword_retriever.k =  10
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.3, 0.7])

In [ ]:
from langchain.retrievers.document_compressors import FlashrankRerank
FlashrankRerank.update_forward_refs()
compressor = FlashrankRerank(top_n=10)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

In [ ]:
from huggingface_hub import InferenceClient
import os
from openai import OpenAI
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

In [ ]:
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  # YOU MUST - Use same embedding function as before
  query_text = 'search query: ' + query_text
  print(query_text)
  response = compression_retriever.invoke(query_text)
  document = []
  for doc in response:
      document.append(doc.page_content)
      docs = "\n".join(document)
      completion = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct",
            messages = [
            {"role": "system", "content": """You are a helpful AI assistant. Your task is to answer the user's questions based on the provided context.
            Strict instructions:
            Language: Always respond in Vietnamese. Absolutely only answer in Vietnamese.
            Reasoning: For any questions involving numbers, quantities, or dates, think step-by-step before providing the final answer.
            Style: Your answer must be concise, yet clear and coherent.
            Here are some examples:
            1. Câu hỏi: Địa chỉ của Trường Đại học công nghệ là gì?
            Câu trả lời: Địa chỉ của Trường Đại học Công nghệ là Nhà E3, 144 Xuân Thủy, quận Cầu Giấy, Hà Nội, Việt Nam.
            2. Câu hỏi: Thời gian hưởng hỗ trợ chi phí học tập cho sinh viên dân tộc thiểu số (không phải 'rất ít người') được quy định là bao nhiêu tháng mỗi năm học?
            Câu trả lời: Thời gian hưởng là 10 tháng/năm học/sinh viên và được xét theo từng học kỳ.
            3. Câu hỏi: Những thách thức chung mà các nhóm sinh viên Trường Đại học Công nghệ thường gặp phải trong quá trình thực hiện nghiên cứu khoa học là gì và họ đã có những giải pháp nào để vượt qua?
            Câu trả lời: Các thách thức chung bao gồm: việc cân bằng giữa thời gian dành cho nghiên cứu và lịch học tập trên lớp, sự cần thiết phải tiếp cận và làm chủ lượng kiến thức mới và chuyên sâu (đặc biệt với các đề tài mang tính liên ngành), thiếu kinh nghiệm thực tế trong việc phát triển một sản phẩm hoàn chỉnh hoặc thực hiện các thí nghiệm phức tạp, cũng như những khó khăn kỹ thuật trong quá trình xây dựng và thử nghiệm sản phẩm hoặc mô hình. Sinh viên đã vượt qua những thách thức này nhờ vào sự kiên trì, nỗ lực không ngừng của bản thân, khả năng làm việc nhóm hiệu quả, sự hỗ trợ và tư vấn kịp thời từ các giảng viên hướng dẫn, việc tích cực tham khảo tài liệu chuyên ngành và học hỏi kinh nghiệm từ các sinh viên khóa trước, cũng như việc xây dựng kế hoạch và quản lý thời gian một cách khoa học.
            4. Câu hỏi: Thời điểm sớm nhất sinh viên có thể đến nhận bằng tốt nghiệp đợt tháng 3 năm 2025 là khi nào?
            Câu trả lời: Sinh viên có thể đến nhận bằng tốt nghiệp đợt tháng 3 năm 2025 bắt đầu từ Thứ hai ngày 28/04/2025.
            """},
            {"role": "user", "content": f"Trả lời câu hỏi này: {query_text}, dựa trên tài liệu sau: {docs}"},
        ],
        )
  return completion.choices[0].message.content
  print(completion.choices[0].message.content)

In [ ]:
import pandas as pd
df_questions = pd.read_csv("/content/drive/MyDrive/rag_ass2/question.csv")
df_questions

,question,answer,level
0,Sinh viên ngành Khoa học Máy tính tại UET được...,"Bên cạnh kiến thức chuyên môn, sinh viên ngành...",Cấp độ 2 - Câu hỏi cần ngữ cảnh
1,Chương trình đào tạo Cử nhân Khoa học Máy tính...,Chương trình Cử nhân Khoa học Máy tính của UET...,Cấp độ 2 - Câu hỏi cần ngữ cảnh
2,"Ngoài việc trở thành lập trình viên, sinh viên...",Sinh viên tốt nghiệp ngành Khoa học Máy tính c...,Cấp độ 2 - Câu hỏi cần ngữ cảnh
3,Tại sao sinh viên ngành Khoa học Máy tính tại ...,Sinh viên ngành Khoa học Máy tính tại UET cần ...,Cấp độ 2 - Câu hỏi cần ngữ cảnh
4,Chương trình đào tạo ngành Khoa học Máy tính c...,Chương trình đào tạo ngành Khoa học Máy tính c...,Cấp độ 2 - Câu hỏi cần ngữ cảnh
...,...,...,...
185,"Trong năm 2025, Đại học Quốc gia Hà Nội đã ký ...","Trong năm 2025, Đại học Quốc gia Hà Nội đã ký ...",Cấp độ 2 - Câu hỏi cần ngữ cảnh
186,Với việc đổi tên vào năm 2022 và di chuyển trụ...,Việc Trung tâm Thư viện và Tri thức số ĐHQGHN ...,Cấp độ 2 - Câu hỏi cần ngữ cảnh
187,"Trong nỗ lực triển khai Nghị quyết 57-NQ/TW, Đ...",Để hiện thực hóa mô hình 'Nhà nước - Nhà trườn...,Cấp độ 2 - Câu hỏi cần ngữ cảnh
188,Các sản phẩm AI tiêu biểu của ĐHQGHN như Hệ th...,"Các sản phẩm AI tiêu biểu của ĐHQGHN, như Hệ t...",Cấp độ 2 - Câu hỏi cần ngữ cảnh


In [ ]:
# prompt: thống kê số lượng theo level

df_questions['level'].value_counts()

,count
level,
Cấp độ 2 - Câu hỏi cần ngữ cảnh,95
Cấp độ 3 - Câu hỏi cần phân tích phức tạp,45
Cấp độ 1 - Câu hỏi cơ bản,25
Cấp độ 4 - Câu hỏi nhạy cảm với thời gian,25


In [ ]:
# prompt: đổi tên cột answer thành reference

df_questions = df_questions.rename(columns={'answer': 'reference'})
print(df_questions.head())

                                            question  \
0  Sinh viên ngành Khoa học Máy tính tại UET được...   
1  Chương trình đào tạo Cử nhân Khoa học Máy tính...   
2  Ngoài việc trở thành lập trình viên, sinh viên...   
3  Tại sao sinh viên ngành Khoa học Máy tính tại ...   
4  Chương trình đào tạo ngành Khoa học Máy tính c...   

                                           reference  \
0  Bên cạnh kiến thức chuyên môn, sinh viên ngành...   
1  Chương trình Cử nhân Khoa học Máy tính của UET...   
2  Sinh viên tốt nghiệp ngành Khoa học Máy tính c...   
3  Sinh viên ngành Khoa học Máy tính tại UET cần ...   
4  Chương trình đào tạo ngành Khoa học Máy tính c...   

                             level  
0  Cấp độ 2 - Câu hỏi cần ngữ cảnh  
1  Cấp độ 2 - Câu hỏi cần ngữ cảnh  
2  Cấp độ 2 - Câu hỏi cần ngữ cảnh  
3  Cấp độ 2 - Câu hỏi cần ngữ cảnh  
4  Cấp độ 2 - Câu hỏi cần ngữ cảnh  


In [ ]:
# prompt: tôi có một file csv question.csv chứa cột question. Hãy dùng query_rag tất cả các question và lưu câu trả lời vào cột mới là answer

# Load the questions from the CSV file
df_questions = pd.read_csv("/content/drive/MyDrive/rag_ass2/question.csv")

# Initialize a new column for answers
df_questions['answer'] = None

# Iterate through each question and query the RAG system
for index, row in df_questions.iterrows():
    question = row['question']
    print(question)
    try:
        answer = query_rag(question)
        print(answer)
        df_questions.loc[index, 'answer'] = answer
        print(f"Processed question {index + 1}: {question[:50]}...") # Print progress
    except Exception as e:
        print(f"Error processing question {index + 1}: {e}")
        df_questions.loc[index, 'answer'] = f"Error: {e}" # Store error message

# Save the updated DataFrame with answers back to a new CSV file
df_questions.to_csv("/content/drive/MyDrive/rag_ass2/question_with_answers.csv", index=False)

print("\nFinished processing questions. Answers saved to question_with_answers.csv")
print(df_questions[['question', 'answer']].head()) # Print the first few rows with questions and answers

In [ ]:
# prompt: thêm cột reference ban đầu nữa

# Load the questions from the CSV file
df = pd.read_csv("/content/drive/MyDrive/rag_ass2/question.csv")

# Rename the 'answer' column to 'reference'
df = df.rename(columns={'answer': 'reference'})

# Initialize a new column for answers
df['answer'] = df_questions['answer']

# Save the updated DataFrame with answers and references back to a new CSV file
df.to_csv("/content/drive/MyDrive/rag_ass2/question_with_answers_and_reference.csv", index=False)


## EVAL

In [ ]:
query_rag("Học phần 'Mạng máy tính' (INT2213) yêu cầu học phần tiên quyết nào?")

search query: Học phần 'Mạng máy tính' (INT2213) yêu cầu học phần tiên quyết nào?


'Học phần "Mạng máy tính" (INT2213) yêu cầu học phần tiên quyết là "Lập trình hướng đối tượng" (INT1008).'

In [ ]:
# prompt: read in that file again, print head
import pandas as pd
#df = pd.read_csv('/content/drive/MyDrive/rag_ass2/question_with_answers_and_reference.csv')
df = pd.read_csv('/content/question_with_answers_and_reference.csv')
df.head()

,question,reference,level,answer
0,Sinh viên ngành Khoa học Máy tính tại UET được...,"Bên cạnh kiến thức chuyên môn, sinh viên ngành...",Cấp độ 2 - Câu hỏi cần ngữ cảnh,Các sinh viên ngành Khoa học Máy tính tại Trườ...
1,Chương trình đào tạo Cử nhân Khoa học Máy tính...,Chương trình Cử nhân Khoa học Máy tính của UET...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,"Dựa trên thông tin có sẵn, Trường Đại học Công..."
2,"Ngoài việc trở thành lập trình viên, sinh viên...",Sinh viên tốt nghiệp ngành Khoa học Máy tính c...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,"Dựa trên tài liệu cung cấp, ngoài việc trở thà..."
3,Tại sao sinh viên ngành Khoa học Máy tính tại ...,Sinh viên ngành Khoa học Máy tính tại UET cần ...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,Sinh viên ngành Khoa học Máy tính tại UET cần ...
4,Chương trình đào tạo ngành Khoa học Máy tính c...,Chương trình đào tạo ngành Khoa học Máy tính c...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,Chương trình đào tạo ngành Khoa học Máy tính c...


In [ ]:
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-pro-preview-05-06", contents="Explain how AI works in a few words"
)
print(response.text)

Okay, here are a few options:

*   **Machines learning from data to make predictions or decisions.**
*   **Software that finds patterns and acts on them.**
*   **Computers learning from examples to perform tasks.**


In [ ]:
from google import genai
from google.genai import types
import time

for row in df.iterrows():
  reference = row[1]['reference']
  answer = row[1]['answer']

  response = client.models.generate_content(
    model="gemini-2.5-pro-preview-05-06",
    config=types.GenerateContentConfig(
        system_instruction="You are a professional language interpreter in Vietnamese. Carefully follow the instruction from the user."),
    contents=f"Your job is to look at a pair of reference and answer. You need to compare the compatibility between them. Please rate the answer with the following rubric:\n0 for \"unsimilar\"\n1 for \"similar\". \nYou are only allowed to output a single digit of number following the above rubrics.\nBEGIN\nReference:{reference}\nAnswer:{answer}. Rating:"
  )
  # Get the rating from the response
  rating = response.text

  # Add the rating to the dataframe
  df.at[row[0], 'Rating'] = rating

# Print the updated dataframe
df.head()


,question,reference,level,answer,Rating
0,Sinh viên ngành Khoa học Máy tính tại UET được...,"Bên cạnh kiến thức chuyên môn, sinh viên ngành...",Cấp độ 2 - Câu hỏi cần ngữ cảnh,Các sinh viên ngành Khoa học Máy tính tại Trườ...,1
1,Chương trình đào tạo Cử nhân Khoa học Máy tính...,Chương trình Cử nhân Khoa học Máy tính của UET...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,"Dựa trên thông tin có sẵn, Trường Đại học Công...",0
2,"Ngoài việc trở thành lập trình viên, sinh viên...",Sinh viên tốt nghiệp ngành Khoa học Máy tính c...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,"Dựa trên tài liệu cung cấp, ngoài việc trở thà...",1
3,Tại sao sinh viên ngành Khoa học Máy tính tại ...,Sinh viên ngành Khoa học Máy tính tại UET cần ...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,Sinh viên ngành Khoa học Máy tính tại UET cần ...,1
4,Chương trình đào tạo ngành Khoa học Máy tính c...,Chương trình đào tạo ngành Khoa học Máy tính c...,Cấp độ 2 - Câu hỏi cần ngữ cảnh,Chương trình đào tạo ngành Khoa học Máy tính c...,1


In [ ]:
# prompt: save df to csv

#df.to_csv('/content/drive/MyDrive/rag_ass2/rated_answers.csv', index=False)
df.to_csv('/content/rated_answers.csv', index=False)

In [ ]:
# prompt: số lượng rating = 1 và rating=0 theo level

# Assuming 'df' is already loaded and has a 'Rating' column and a 'level' column
# If 'level' column doesn't exist, you'll need to load or create it
df = pd.read_csv('/content/drive/MyDrive/rag_ass2/rated_answers.csv')
# Convert 'Rating' to numeric, handling potential errors
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows where 'Rating' is NaN (if any conversion errors occurred)
df.dropna(subset=['Rating'], inplace=True)

# Convert 'Rating' to integer
df['Rating'] = df['Rating'].astype(int)


# Group by 'level' and 'Rating' and count the occurrences
rating_counts_by_level = df.groupby(['level', 'Rating']).size().unstack(fill_value=0)

# Rename columns for clarity
rating_counts_by_level.rename(columns={0: 'count_rating_0', 1: 'count_rating_1'}, inplace=True)

# Print the result
rating_counts_by_level


Rating,count_rating_0,count_rating_1
level,,
Cấp độ 1 - Câu hỏi cơ bản,3,22
Cấp độ 2 - Câu hỏi cần ngữ cảnh,25,70
Cấp độ 3 - Câu hỏi cần phân tích phức tạp,26,19
Cấp độ 4 - Câu hỏi nhạy cảm với thời gian,9,16
